## Summing all the prime numbers below a given number

In [ ]:
import sys

if len(sys.argv) > 1:
    number = int(sys.argv[1])
else:
    print ("Debe introducir un valor entero")

In [6]:
import time

# Simple code

def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

#number  = 2_500_000
suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print ("-------------- CÓDIGO ORIGINAL --------------")
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 4.786287625630696
The prime sum below  2500000 is  219697708195  and the time taken is 4.61 s ± 29.8 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


#### Apartado a)

In [2]:
import time
from numba import njit

# Simple code
@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result


#number = 2_500_000
suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print ("-------------- NUMBA SECUENCIAL --------------")
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 0.6685638427734375
The prime sum below  2500000 is  219697708195  and the time taken is 577 ms ± 2.84 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


#### Apartado b)

In [7]:
import time
from numba import njit
from multiprocessing import Pool

# Simple code
@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x


def sum_primes_partes(rango):
    inicio, fin = rango
    result = 0
    for i in range(inicio, fin):
        result += if_prime(i)
    return result


#Obligatorio poner lo siguiente para que no cree procesos infinitos
if __name__ == "__main__":
    
    #Introducimos parámetros
    #number = 2_500_000
    suma = 0
    N = 3 # number of loops
    n_cores = 4 #nº cores, lo ponemos aquí para que esté más visible

    
    #Ejecutamos Numba para que compile
    sum_primes_partes((0, 10)) #Función inicial

    #Dividimos en funcion del num procesadores
    seccion = number // n_cores
    secciones = [] #Inicializamos lista vacía

    #Vamos añadiendo a una lista los rangos de numeros obtenidos
    for i in range (n_cores):
        inicio = i * seccion
        if i == n_cores - 1 :  
            fin = number
        else:
            fin = (i + 1) * seccion
        secciones.append((inicio, fin))
    
    #Paralelización 
    start = time.time()
    for _ in range(N):
        with Pool(n_cores) as p: #4 procesadores
            sumas_parciales = p.map(sum_primes_partes, secciones) #mapa, la misma función sobre las 4 secciones generadas
        suma = sum(sumas_parciales) #Obtenemos la suma total
    stop = time.time()
    tiempo = (stop - start) / N
    
    
    #Imprimimos resultados
    print ("-------------- MULTIPROCESSING CON POOL --------------")
    print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)
    
    %timeit -r 2 -q sum_primes_partes((0, number))
    suma = sum_primes_partes((0, number))
    print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

The prime sum below  2500000 is  219697708195  and the time taken is 0.18224438031514487
The prime sum below  2500000 is  219697708195  and the time taken is 0.18224438031514487


#### Apartado c)

In [1]:
import time
import numpy as np
from numba import njit, prange

# Simple code
@njit
def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit(parallel=True)
def sum_primes_prange(x): 
    results = np.zeros(x, dtype=np.int64) #Aquí es importante definir distintos results para paralelizar. nº enteros
    for i in prange(x): #paraleliza
        results[i] = if_prime(i) 
    return results.sum()

#number = 2_500_000
suma = 0
N = 3 # number of loops

#Compilamos Numba
sum_primes_prange(10)

start = time.time()
for i in range(N):
    suma = sum_primes_prange(number)
stop = time.time()
tiempo = (stop - start) / N

print ("-------------- NUMBA CON PRANGE --------------")
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes_prange(number)
suma = sum_primes_prange(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


/usr/local/lib/python3.13/site-packages/numba/core/typed_passes.py:338: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../tmp/ipykernel_281021/347970947.py", line 20:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


NameError: name 'number' is not defined

# RESULTADOS

## CPUS = 1
### Ejecutando con numero: 1000000
#### -------------- CÓDIGO ORIGINAL --------------
The prime sum below  1000000 is  37550402023  and the time taken is 2.2775729497273765

The prime sum below  1000000 is  37550402023  and the time taken is 2.21 s ± 1.76 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- NUMBA SECUENCIAL --------------
The prime sum below  1000000 is  37550402023  and the time taken is 1.8304036458333333

The prime sum below  1000000 is  37550402023  and the time taken is 370 ms ± 57.7 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- MULTIPROCESSING CON POOL --------------
The prime sum below  1000000 is  37550402023  and the time taken is 0.10978976885477702

The prime sum below  1000000 is  37550402023  and the time taken is 0.10978976885477702
#### -------------- NUMBA CON PRANGE --------------
The prime sum below  1000000 is  37550402023  and the time taken is 0.11464285850524902

The prime sum below  1000000 is  37550402023  and the time taken is 115 ms ± 3.68 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)



### Ejecutando con numero: 10000000
#### -------------- CÓDIGO ORIGINAL --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 59.53139511744181

The prime sum below  10000000 is  3203324994356  and the time taken is 57.4 s ± 14.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- NUMBA SECUENCIAL --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 5.310391028722127

The prime sum below  10000000 is  3203324994356  and the time taken is 4.98 s ± 6.74 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- MULTIPROCESSING CON POOL --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 1.4243816534678142

The prime sum below  10000000 is  3203324994356  and the time taken is 1.4243816534678142
#### -------------- NUMBA CON PRANGE --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 2.870138645172119

The prime sum below  10000000 is  3203324994356  and the time taken is 2.87 s ± 119 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)


## -------------------------------------------------------------------------------------------
## CPUS = 2
### Ejecutando con numero: 1000000
####  -------------- CÓDIGO ORIGINAL --------------
The prime sum below  1000000 is  37550402023  and the time taken is 2.2615036169687905

The prime sum below  1000000 is  37550402023  and the time taken is 2.19 s ± 1.38 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- NUMBA SECUENCIAL --------------
The prime sum below  1000000 is  37550402023  and the time taken is 0.8340488274892172

The prime sum below  1000000 is  37550402023  and the time taken is 344 ms ± 190 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- MULTIPROCESSING CON POOL --------------
The prime sum below  1000000 is  37550402023  and the time taken is 0.11337836583455403

The prime sum below  1000000 is  37550402023  and the time taken is 0.11337836583455403
#### -------------- NUMBA CON PRANGE --------------
The prime sum below  1000000 is  37550402023  and the time taken is 0.07057499885559082

The prime sum below  1000000 is  37550402023  and the time taken is 70.5 ms ± 3.09 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)



### Ejecutando con numero: 10000000
#### -------------- CÓDIGO ORIGINAL --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 59.11098186175028

The prime sum below  10000000 is  3203324994356  and the time taken is 57.5 s ± 88.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- NUMBA SECUENCIAL --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 4.833821376164754

The prime sum below  10000000 is  3203324994356  and the time taken is 4.86 s ± 413 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- MULTIPROCESSING CON POOL --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 1.418907642364502

The prime sum below  10000000 is  3203324994356  and the time taken is 1.418907642364502
#### -------------- NUMBA CON PRANGE --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 1.788112958272298

The prime sum below  10000000 is  3203324994356  and the time taken is 1.79 s ± 83.2 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)


## -------------------------------------------------------------------------------------------
## CPUS = 4
### Ejecutando con numero: 1000000
#### -------------- CÓDIGO ORIGINAL --------------
The prime sum below  1000000 is  37550402023  and the time taken is 2.292612314224243

The prime sum below  1000000 is  37550402023  and the time taken is 2.21 s ± 859 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- NUMBA SECUENCIAL --------------
The prime sum below  1000000 is  37550402023  and the time taken is 1.7995010217030842

The prime sum below  1000000 is  37550402023  and the time taken is 346 ms ± 20.6 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- MULTIPROCESSING CON POOL --------------
The prime sum below  1000000 is  37550402023  and the time taken is 0.12223776181538899

The prime sum below  1000000 is  37550402023  and the time taken is 0.12223776181538899
#### -------------- NUMBA CON PRANGE --------------
The prime sum below  1000000 is  37550402023  and the time taken is 0.037687222162882485

The prime sum below  1000000 is  37550402023  and the time taken is 37.6 ms ± 2.18 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)




### Ejecutando con numero: 10000000
#### -------------- CÓDIGO ORIGINAL --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 60.927420139312744

The prime sum below  10000000 is  3203324994356  and the time taken is 57.6 s ± 80.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- NUMBA SECUENCIAL --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 4.8626002470652265

The prime sum below  10000000 is  3203324994356  and the time taken is 4.85 s ± 521 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- MULTIPROCESSING CON POOL --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 1.4160192012786865

The prime sum below  10000000 is  3203324994356  and the time taken is 1.4160192012786865
#### -------------- NUMBA CON PRANGE --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 0.9567975203196207

The prime sum below  10000000 is  3203324994356  and the time taken is 957 ms ± 12.5 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)


## -------------------------------------------------------------------------------------------
## CPUS = 8
### Ejecutando con numero: 1000000
#### -------------- CÓDIGO ORIGINAL --------------
The prime sum below  1000000 is  37550402023  and the time taken is 2.2623933951059976

The prime sum below  1000000 is  37550402023  and the time taken is 2.2 s ± 2.07 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- NUMBA SECUENCIAL --------------
The prime sum below  1000000 is  37550402023  and the time taken is 0.6258850892384847

The prime sum below  1000000 is  37550402023  and the time taken is 344 ms ± 31.9 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- MULTIPROCESSING CON POOL --------------
The prime sum below  1000000 is  37550402023  and the time taken is 0.10927549997965495

The prime sum below  1000000 is  37550402023  and the time taken is 0.10927549997965495
#### -------------- NUMBA CON PRANGE --------------
The prime sum below  1000000 is  37550402023  and the time taken is 0.01943548520406087

The prime sum below  1000000 is  37550402023  and the time taken is 19.5 ms ± 210 μs per loop (mean ± std. dev. of 2 runs, 100 loops each)





### Ejecutando con numero: 10000000
#### -------------- CÓDIGO ORIGINAL --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 59.37160348892212

The prime sum below  10000000 is  3203324994356  and the time taken is 57.6 s ± 80.5 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- NUMBA SECUENCIAL --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 4.878125031789144

The prime sum below  10000000 is  3203324994356  and the time taken is 4.85 s ± 1.33 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
#### -------------- MULTIPROCESSING CON POOL --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 1.4279817740122478

The prime sum below  10000000 is  3203324994356  and the time taken is 1.4279817740122478
#### -------------- NUMBA CON PRANGE --------------
The prime sum below  10000000 is  3203324994356  and the time taken is 0.4926036198933919

The prime sum below  10000000 is  3203324994356  and the time taken is 493 ms ± 120 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)


# Interpretación de los resultados (apartado e)

#### -------------- CÓDIGO ORIGINAL --------------
Para el código original, destacamos unos tiempos de en torno a **2,27s para 10^6**; mientras que cuando calculamos para **10^7, los tiempos son de 59,5s**. 

Como es de esperar, el código original lleva a cabo la identificación de forma secuencial de todos los números primos, los guarda, y después lleva a cabo la suma de todos. Es normal que este proceso sea el que más tiempo conlleve al no paralelizar y usar Python "puro".

#### -------------- NUMBA SECUENCIAL --------------
Al emplear numba secuencial (@njit), el cálculo se acelera mucho, pues se lleva a cabo la compilación y se elimina gran cantidad del tiempo por la forma de actuar de Python, pues es un idioma interpretado.

Así, los tiempos para **10^6 se reducen a 0,83s** (X2,7); mientras que en **10^7 se consigue reducir a una media de hasta 4,83s** (X12,3). Como vemos, el tiempo de cálculo disminuye muchísimo al tratar con un valor de 10^7.

#### -------------- MULTIPROCESSING CON POOL --------------
Cuando se realiza paralelización, lo que hacemos es dividir los números primos que se han obtenido en el número de CPUs que queramos usar y, después, se reparten entre ellas, permitiendo reducir muchísimo el tiempo de carga al realizar los cálculos de maneras independientes y luego simplemente sumar los cálculos realizados por cada uno de los CPUs.

En este caso, al realizar multiprocessing con Pool, los tiempos se mantienen en **0,11s y 1,42s para 10^6 y 10^7**, respectivamente.

#### -------------- NUMBA CON PRANGE --------------
prange permite paralelizar los bucles dentro de Numba, sin crear procesos externos como ocurre con Pool de multiprocessing. Esto permite una comunicación entre hilos más eficiente, mejorando aún más la velocidad de compilación y obteniendo los mejores tiempos:
**- Tiempo medio 10^6: 0,07s**
**- Tiempo medio 10^7: 1,79s**

#### Paralelización
Cabe destacar que cuando comparamos los diferentes tiempos obtenidos al emplear distintas CPUs, no se reduce el tiempo en los dos primeros casos al emplear únicamente una CPU, como es de esperar al emplear tareas secuenciales.

De igual forma, vemos que para **multiprocessing con pool**, los tiempos no se reducen al aumentar el número de CPUs. Esto se debe al modo de actuar de Pool, pues lleva a cabo la generación de procesos externos donde se tiene en cuenta la subida de los procesos, los cálculos independientes de los diferentes primos... El problema de esto es que cada proceso ocupa un espacio en memoria y la comunicación entre diferentes espacios empleando este proceso no es muy efectivo, por lo que no siempre el incremento de CPUs se corresponde con una reducción del tiempo.

Con **numba y prange** sí que vemos una reducción en los tiempos de carga al incrementar los procesadores (0,11; 0,07; 0,03; 0,019s), problema solucionado con una mejor comunicación por medio de los hilos.